In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [16]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 5, 27, 19, 48, 52),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [17]:
print(db.query('''SELECT * FROM STOCKS ;'''))

[('GME', datetime.datetime(2021, 5, 14, 1, 43, 41))]


In [ ]:
indexes = db.query('''SELECT POST_ID FROM POSTS;''')

In [35]:
db.query("SELECT TITLE FROM POSTS WHERE POST_ID='{}'".format(indexes[0][0]))[0][0]

"I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO"